In [6]:
import pandas as pd
import numpy as np
import geopandas as gpd
pd.set_option('display.max_columns', None)

In [45]:
dataset = pd.read_csv("/Users/ferhatbolukbas/Desktop/foodstyle/dataset.csv")
capitals = pd.read_csv("/Users/ferhatbolukbas/Desktop/foodstyle/europe_capitals_population_and_area.csv")
paris_bounds = pd.read_json("/Users/ferhatbolukbas/Desktop/foodstyle/paris_bounding_polygon.json")

/var/folders/vw/rpcvp4qn2m53bgw4c2xkmmr40000gn/T/ipykernel_23129/3333871273.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("/Users/ferhatbolukbas/Desktop/foodstyle/dataset.csv")


### Data cleaning

In [8]:
dataset.head()

,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,longitude,claimed,awards,popularity_detailed,popularity_generic,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,original_open_hours,open_days_per_week,open_hours_per_week,working_shifts_per_week,avg_rating,total_reviews_count,default_language,reviews_count_in_default_language,excellent,very_good,average,poor,terrible,food,service,value,atmosphere,keywords
0,g10001637-d10002227,Le 147,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"10 Maison Neuve, 87510 Saint-Jouvent France",45.961674,1.169131,Claimed,NaN,#1 of 2 Restaurants in Saint-Jouvent,#1 of 2 places to eat in Saint-Jouvent,"Cheap Eats, French",€,NaN,"Lunch, Dinner",French,NaN,"Reservations, Seating, Wheelchair Accessible, ...",N,N,N,NaN,NaN,NaN,NaN,4.0,36.0,English,2.0,2.0,0.0,0.0,0.0,0.0,4.0,4.5,4.0,NaN,NaN
1,g10001637-d14975787,Le Saint Jouvent,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Ha...",France,Nouvelle-Aquitaine,Haute-Vienne,Saint-Jouvent,"16 Place de l Eglise, 87510 Saint-Jouvent France",45.957040,1.205480,Unclaimed,NaN,#2 of 2 Restaurants in Saint-Jouvent,#2 of 2 places to eat in Saint-Jouvent,Cheap Eats,€,NaN,NaN,NaN,NaN,NaN,N,N,N,NaN,NaN,NaN,NaN,4.0,5.0,All languages,5.0,2.0,2.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,g10002858-d4586832,Au Bout du Pont,"[""Europe"", ""France"", ""Centre-Val de Loire"", ""B...",France,Centre-Val de Loire,Berry,Rivarennes,"2 rue des Dames, 36800 Rivarennes France",46.635895,1.386133,Claimed,NaN,#1 of 1 Restaurant in Rivarennes,#1 of 1 places to eat in Rivarennes,"Cheap Eats, French, European",€,NaN,"Dinner, Lunch, Drinks","French, European",NaN,"Reservations, Seating, Table Service, Wheelcha...",N,N,N,NaN,NaN,NaN,NaN,5.0,13.0,English,4.0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,g10002986-d3510044,Le Relais de Naiade,"[""Europe"", ""France"", ""Nouvelle-Aquitaine"", ""Co...",France,Nouvelle-Aquitaine,Correze,Lacelle,"9 avenue Porte de la Correze 19170, 19170 Lace...",45.642610,1.824460,Claimed,NaN,#1 of 1 Restaurant in Lacelle,#1 of 1 places to eat in Lacelle,"Cheap Eats, French",€,NaN,"Lunch, Dinner",French,NaN,"Reservations, Seating, Serves Alcohol, Table S...",N,N,N,NaN,NaN,NaN,NaN,4.0,34.0,English,1.0,1.0,0.0,0.0,0.0,0.0,4.5,4.5,4.5,NaN,NaN
4,g10022428-d9767191,Relais Du MontSeigne,"[""Europe"", ""France"", ""Occitanie"", ""Aveyron"", ""...",France,Occitanie,Aveyron,Saint-Laurent-de-Levezou,"route du Montseigne, 12620 Saint-Laurent-de-Le...",44.208860,2.960470,Unclaimed,NaN,#1 of 1 Restaurant in Saint-Laurent-de-Levezou,#1 of 1 places to eat in Saint-Laurent-de-Levezou,"Mid-range, French",€€-€€€,NaN,"Lunch, Dinner",French,NaN,"Reservations, Seating, Wheelchair Accessible, ...",N,N,N,NaN,NaN,NaN,NaN,4.5,11.0,All languages,11.0,4.0,7.0,0.0,0.0,0.0,4.5,4.5,4.5,NaN,NaN


In [12]:
# Identify the columns with mixed data types:

columns_data_types = {}

# Iterate through each column in the dataframe
for column in dataset.columns:
    # Get the unique data types for the column
    unique_data_types = dataset[column].apply(lambda x: type(x)).unique()
    # If there are more than one unique data types, add the column name and its data types to the dictionary
    if len(unique_data_types) > 1:
        columns_data_types[column] = unique_data_types
columns_data_types

{'region': array([<class 'str'>, <class 'float'>], dtype=object),
 'province': array([<class 'str'>, <class 'float'>], dtype=object),
 'city': array([<class 'str'>, <class 'float'>], dtype=object),
 'claimed': array([<class 'str'>, <class 'float'>], dtype=object),
 'awards': array([<class 'float'>, <class 'str'>], dtype=object),
 'popularity_detailed': array([<class 'str'>, <class 'float'>], dtype=object),
 'popularity_generic': array([<class 'str'>, <class 'float'>], dtype=object),
 'top_tags': array([<class 'str'>, <class 'float'>], dtype=object),
 'price_level': array([<class 'str'>, <class 'float'>], dtype=object),
 'price_range': array([<class 'float'>, <class 'str'>], dtype=object),
 'meals': array([<class 'str'>, <class 'float'>], dtype=object),
 'cuisines': array([<class 'str'>, <class 'float'>], dtype=object),
 'special_diets': array([<class 'float'>, <class 'str'>], dtype=object),
 'features': array([<class 'str'>, <class 'float'>], dtype=object),
 'original_open_hours': arra

In [14]:
# For each column, count the number of rows per data type:
for column, data_types in columns_data_types.items():
    print(f'Column: {column}')
    for data_type in data_types:
        count = dataset[column].apply(lambda x: type(x) == data_type).sum()
        print(f'{data_type}: {count}')

Column: region
<class 'str'>: 1033074
<class 'float'>: 50323
Column: province
<class 'str'>: 742765
<class 'float'>: 340632
Column: city
<class 'str'>: 682712
<class 'float'>: 400685
Column: claimed
<class 'str'>: 1081555
<class 'float'>: 1842
Column: awards
<class 'float'>: 820264
<class 'str'>: 263133
Column: popularity_detailed
<class 'str'>: 988409
<class 'float'>: 94988
Column: popularity_generic
<class 'str'>: 985605
<class 'float'>: 97792
Column: top_tags
<class 'str'>: 972763
<class 'float'>: 110634
Column: price_level
<class 'str'>: 806192
<class 'float'>: 277205
Column: price_range
<class 'float'>: 779070
<class 'str'>: 304327
Column: meals
<class 'str'>: 635347
<class 'float'>: 448050
Column: cuisines
<class 'str'>: 914294
<class 'float'>: 169103
Column: special_diets
<class 'float'>: 743141
<class 'str'>: 340256
Column: features
<class 'str'>: 317407
<class 'float'>: 765990
Column: original_open_hours
<class 'float'>: 489565
<class 'str'>: 593832
Column: default_language
<c

Would removing missing values solve the mixed data type problem?


Removing missing values will not necessarily solve the mixed data type problem.
The issue is that the data in the column is not consistent and has multiple types.
Removing missing values will only change the number of rows per data type but 
the issue of inconsistent data types will still persist. 
A better approach would be to either convert the data to a 
consistent type or exclude the column from analysis.

In [ ]:
### Data understanding

In [15]:
#Are the review columns correlated with the rating columns?

review_columns = ["excellent", "very_good", "average", "poor", "terrible", "total_reviews_count", "reviews_count_in_default_language"]
rating_columns = ["food", "service", "value", "atmosphere", "avg_rating"]

# Correlation matrix
corr_matrix = dataset[review_columns + rating_columns].corr()

# Print the correlation matrix
corr_matrix

                                   excellent  very_good   average      poor  \
excellent                           1.000000   0.842964  0.733120  0.632904   
very_good                           0.842964   1.000000  0.928148  0.825488   
average                             0.733120   0.928148  1.000000  0.934195   
poor                                0.632904   0.825488  0.934195  1.000000   
terrible                            0.497784   0.672637  0.791354  0.878500   
total_reviews_count                 0.716426   0.710900  0.634119  0.539704   
reviews_count_in_default_language   0.959361   0.947613  0.887308  0.805920   
food                                0.128760   0.016350 -0.060098 -0.103778   
service                             0.134771   0.020838 -0.051280 -0.093287   
value                               0.099463  -0.010068 -0.083435 -0.125787   
atmosphere                          0.167081   0.067123 -0.014837 -0.057878   
avg_rating                          0.093861   0.011

In [16]:
#Are vegetarian-friendly restaurants better than non-vegetarian ones?

#Create a new column for restaurants with a binary value (1 for vegetarian-friendly and 0 for non-vegetarian-friendly)
dataset["vegetarian_friendly_binary"] = dataset["vegetarian_friendly"].apply(lambda x: 1 if x == "Yes" else 0)

# Compare the average rating for vegetarian-friendly and non-vegetarian-friendly restaurants
vegetarian_friendly_avg_rating = dataset[dataset["vegetarian_friendly_binary"] == 1]["avg_rating"].mean()
non_vegetarian_friendly_avg_rating = dataset[dataset["vegetarian_friendly_binary"] == 0]["avg_rating"].mean()

print(f'Average rating for vegetarian-friendly restaurants: {vegetarian_friendly_avg_rating}')
print(f'Average rating for non-vegetarian-friendly restaurants: {non_vegetarian_friendly_avg_rating}')

Average rating for vegetarian-friendly restaurants: nan
Average rating for non-vegetarian-friendly restaurants: 4.035942847356148


In [21]:
dataset["price_level"].unique()

array(['€', '€€-€€€', nan, '€€€€'], dtype=object)

In [57]:
#Are there any significantly more expensive cuisines?
# Create a new column for the average price level of a cuisine - note: price_level to int

#dicti = {'€':'1', '€€-€€€':'3', '€€€€':'2', 'nan':np.nan}

#dataset["price_level"] = dataset["price_level"].replace(dicti)
dataset["price_level"] = pd.to_numeric(dataset["price_level"])

avg_cuisines_prices = dataset.groupby('cuisines').agg({'price_level':'mean'}).reset_index().dropna().sort_values(by = 'price_level')
avg_cuisines_prices.tail(25)

,cuisines,price_level
41512,"Fusion, Healthy, Indian",3.0
41513,"Fusion, Healthy, International",3.0
41514,"Fusion, Healthy, Italian, Seafood, Mediterrane...",3.0
41515,"Fusion, Healthy, Japanese",3.0
41517,"Fusion, Healthy, Mediterranean, Spanish",3.0
41518,"Fusion, Healthy, Mediterranean, Wine Bar",3.0
41519,"Fusion, Healthy, Polish, Mediterranean",3.0
41520,"Fusion, Healthy, Portuguese, African, Bar",3.0
41523,"Fusion, Healthy, Soups, Dining bars",3.0
41524,"Fusion, Healthy, Spanish",3.0


In [ ]:
### Business-specific

- In the assets directory, you will see a very small dataset called europe_capitals_population_and_area.csv. A gluten-free restaurant wants to open a new restaurant in a European capital where gluten-free restaurants are underrepresented. Assuming there are no other factors, except population and gluten-free restaurant density, what would be the top 5 capitals to open that restaurant?
- Think and propose a couple of other ways this dataset could be used to help businesses.


In [ ]:
dataset_gl = dataset.loc[dataset['gluten_free']=='Y'].groupby(['city', 'country']).size().reset_index()
dataset_gl.columns = ['city', 'country', 'num_of_gluten_restorant']

final_data = dataset_gl.merge(capitals, on = ['city', 'country'], how = 'left').dropna()

final_data['gluten_free_density'] = final_data['num_of_gluten_restorant'] / final_data['Population']
final_data.sort_values(by=['gluten_free_density'], inplace=True)
final_data

### Bonus

In [ ]:
In the assets directory, you will see a file called paris_bounding_polygon.json.
This contains a list of latitude and longitude coordinates that define a polygon
that is considered to represent the Paris city area. For simplicity, 
we assume the population distribution is uniform in the Paris city area. 

An Italian restaurant wants to open a restaurant in Paris in a zone where there are
the fewest Italian restaurants. What is the best location to open the restaurant 
(the answer can be a single point or a bounding box/polygonal region depending on the implementation)?

In [68]:
italy_cuisines_in_paris = dataset.loc[dataset['city']=='Paris'].loc[dataset['cuisines'].str.contains('Italian')==True]
italy_cuisines_in_paris

,restaurant_link,restaurant_name,original_location,country,region,province,city,address,latitude,longitude,claimed,awards,popularity_detailed,popularity_generic,top_tags,price_level,price_range,meals,cuisines,special_diets,features,vegetarian_friendly,vegan_options,gluten_free,original_open_hours,open_days_per_week,open_hours_per_week,working_shifts_per_week,avg_rating,total_reviews_count,default_language,reviews_count_in_default_language,excellent,very_good,average,poor,terrible,food,service,value,atmosphere,keywords
38542,g187147-d10000085,La Cucina,"[""Europe"", ""France"", ""Ile-de-France"", ""Paris""]",France,Ile-de-France,NaN,Paris,"36 boulevard Garibaldi, 75015 Paris France",48.846660,2.305100,Unclaimed,NaN,#13163 of 15478 Restaurants in Paris,#13978 of 18480 places to eat in Paris,Italian,NaN,NaN,NaN,Italian,NaN,NaN,N,N,N,NaN,NaN,NaN,NaN,3.5,2.0,All languages,2.0,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
38554,g187147-d10027353,L'inconnu,"[""Europe"", ""France"", ""Ile-de-France"", ""Paris""]",France,Ile-de-France,NaN,Paris,"4 rue Pierre Leroux, 75007 Paris France",48.850185,2.318649,Claimed,Michelin 2021 for The Michelin Plate: Good coo...,#1159 of 15478 Restaurants in Paris,#1229 of 18480 places to eat in Paris,"Fine Dining, Italian, European",2.0,NaN,"Lunch, Dinner","Italian, European",NaN,"Reservations, Seating, Wheelchair Accessible, ...",N,N,N,NaN,NaN,NaN,NaN,4.5,127.0,English,26.0,19.0,5.0,1.0,0.0,1.0,4.5,4.5,4.0,NaN,NaN
38555,g187147-d10027589,Soyouz,"[""Europe"", ""France"", ""Ile-de-France"", ""Paris""]",France,Ile-de-France,NaN,Paris,"11 rue de l Esperance, 75013 Paris France",48.827220,2.348650,Unclaimed,NaN,#9028 of 15478 Restaurants in Paris,#9635 of 18480 places to eat in Paris,"Cheap Eats, Italian, Pizza",1.0,NaN,NaN,"Italian, Pizza",NaN,"Reservations, Seating, Wheelchair Accessible",N,N,N,NaN,NaN,NaN,NaN,4.0,15.0,English,1.0,0.0,0.0,0.0,0.0,1.0,4.0,4.0,4.0,NaN,NaN
38569,g187147-d10034691,Marius,"[""Europe"", ""France"", ""Ile-de-France"", ""Paris""]",France,Ile-de-France,NaN,Paris,"186 rue Ordener, 75018 Paris France",48.894110,2.333324,Claimed,NaN,#7473 of 15478 Restaurants in Paris,#7957 of 18480 places to eat in Paris,"Cheap Eats, Italian",1.0,€10-€20,"Dinner, Lunch, After-hours, Drinks",Italian,NaN,NaN,N,N,N,"{""Mon"": [""12:00-15:00"", ""18:30-23:00""], ""Tue"":...",7.0,52.5,14.0,4.5,16.0,English,2.0,2.0,0.0,0.0,0.0,0.0,4.5,5.0,5.0,NaN,NaN
38572,g187147-d10039974,PI HOUR Batignolles,"[""Europe"", ""France"", ""Ile-de-France"", ""Paris""]",France,Ile-de-France,NaN,Paris,"16 rue Jacquemont, 75017 Paris France",48.887800,2.322460,Claimed,Certificate of Excellence 2017,#3982 of 15478 Restaurants in Paris,#4204 of 18480 places to eat in Paris,"Mid-range, Italian, Pizza, Vegetarian Friendly",3.0,€10-€25,NaN,"Italian, Pizza",Vegetarian Friendly,NaN,Y,N,N,NaN,NaN,NaN,NaN,4.5,47.0,All languages,47.0,34.0,11.0,1.0,0.0,1.0,4.5,4.5,4.5,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56612,g187147-d9826837,Pizza Verdi,"[""Europe"", ""France"", ""Ile-de-France"", ""Paris""]",France,Ile-de-France,NaN,Paris,"40 rue du Faubourg Saint Denis, 75010 Paris Fr...",48.871387,2.353561,Claimed,NaN,#11044 of 15476 Restaurants in Paris,#11759 of 18480 places to eat in Paris,"Cheap Eats, Italian, Pizza, Vegetarian Friendly",1.0,€9-€17,NaN,"Italian, Pizza",Vegetarian Friendly,NaN,Y,N,N,NaN,NaN,NaN,NaN,3.5,24.0,English,2.0,0.0,0.0,2.0,0.0,0.0,3.5,3.5,4.0,NaN,NaN
56636,g187147-d9849038,66 Yeast,"[""Europe"", ""France"", ""Ile-de-France"", ""Paris""]",France,Ile-de-France,NaN,Paris,"66 rue d Hauteville, 75010 Paris France",48.874920,2.350872,Unclaimed,NaN,#7702 of 15476 Restaurants in Paris,#8207 of 18480 places to eat in Paris,"Mid-range, Italian, Pizza",3.0,€8-€20,Dinner,"Pizza, Italian",NaN,NaN,N,N,N,NaN,NaN,NaN,NaN,4.5,14.0,English,2.0,1.0,1.0,0.0,0.0,0.0,4.5,5.0,4.5,NaN,NaN
56649,g187147-d9872703

In [73]:
boundries = paris_bounds.values

In [91]:
import pandas as pd
import plotly.express as px
fig = px.density_mapbox(italy_cuisines_in_paris, lat='latitude', lon='longitude', z='price_level', radius=10,
                        center=dict(lat=0, lon=180), zoom=0,
                        mapbox_style="stamen-terrain")
fig.update_layout(
    mapbox = {
        'style': "stamen-terrain",
        'center': { 'lon': 2.3428763772885635 , 'lat': 48.85662216690112},
        'zoom': 12, 'layers': [{
            'source': {
                'type': "FeatureCollection",
                'features': [{
                    'type': "Feature",
                    'geometry': {
                        'type': "Polygon",
                        'coordinates':  [boundries]
                    }
                }]
            },
            'type': "line", 'below': "traces", 'color': "red"}]},
    margin = {'l':0, 'r':0, 'b':0, 't':0})
fig.show()